# Red Neuronal Profunda (DNN) para clasificación MNIST

Aplicaremos todos nuestros conocimientos para crear una DNN, frecuentemente llamada también una Artificial Neural Network (ANN).  El problema que vamos a trabajar se conoce como el "Hola Mundo" del aprendizaje profundo porque para la mayoría de estudiantes este es el primer algoritmo de aprendizaje profundo que ven. 

El conjunto de datos se llama MNIST y se refiere al reconocimiento de dígitos escritos a mano.  Pueden encontrar más información en el sitio web de Yann LeCun (Director of AI Research, Facebook).  El es uno de los pioneros de todo este tema, así como de otras metodologías más complejas como las Redes Neurales Convolucionales (CNN) que se utilizan hoy día.

El conjunto de datos tiene 70,000 imágenes (28x28 pixels) de dígitos escritos a mano (1 dígito por imagen).

La meta es escribir un algoritmo que detecta qué dígito ha sido escrito.  Como solo hay 10 dígitos (0 al 9), este es un problema de clasificación con 10 clases.

Nuestra meta será construir una RN con 2 capas escondidas.

## Importar los paquetes relevantes

TensorFlow incluye un proveedor de datos de MNIST que utilizaremos acá.  Viene con el módulo **"tensorflow.keras.datasets"**. 

In [3]:
%pip install tensorflow

  Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached ml_dtypes-0.4.0-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.4-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.65.4-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached tensorboard-2.17.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.4.1-py3-none-any.whl.metadata (5.8 kB)
  Using cache

ERROR: Could not install packages due to an OSError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'c:\\Python312\\Scripts\\markdown_py.exe' -> 'c:\\Python312\\Scripts\\markdown_py.exe.deleteme'



In [6]:
import numpy as np
import tensorflow as tf

La siguiente instrucción, cuando se corre por primera vez, descarga el conjunto de datos en lo indicado por el parámetro path, relativo a  ~/.keras/datasets).  Como si se hubiera ejecutado Lo siguiente:

tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

luego separa los datos en un conjunto para entrenamiento y otro para pruebas.

Si se ejecuta más de una vez, ya no descarga el archivo.

In [7]:
(X_entreno, y_entreno), (X_prueba, y_prueba) = tf.keras.datasets.mnist.load_data()
assert X_entreno.shape == (60000, 28, 28)
assert X_prueba.shape == (10000, 28, 28)
assert y_entreno.shape == (60000,)
assert y_prueba.shape == (10000,)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## Datos

Esta sección es donde pre-procesaremos nuestros datos.

Por default, TF2 tiene conjuntos de datos de entrenamiento y de prueba, pero no tiene un conjunto de validación, por lo que debemos dividirlo por nuestra cuenta

Lo haremos del mismo tamaño que el conjunto de prueba

In [8]:
num_obs_validacion = y_prueba.shape[0]

Usaremos una variable dedicada para el número de muestras de prueba

In [9]:
num_obs_prueba = y_prueba.shape[0]

Generalmente preferimos "normalizar" nuestros datos en alguna forma para que el resultado sea numéricamente más estable.  En este caso simplemente preferimos tener entradas entre 0 y 1, por lo que definimos una función, que reciba la imagen MNIST.

Como los posibles valores de las entradas son entre 0 y 255 (256 posibles tonos de gris), al dividirlos por 255 obtenemos el resultado deseado.

In [10]:
X_entreno_normalizado = X_entreno / 255

Finalmente, normalizaremos y convertiremos los datos de pruebas en tandas.  Los normalizamos para que tengan la misma magnitud que los datos de entrenamiento y validación.

No hay necesidad de "barajearlo" ya que no estaremos entrenando con los datos de prueba.  Habra una sola tanda, igual al tamaño de los datos de prueba.

In [11]:
X_prueba_normalizado = X_prueba / 255

Una vez se han "normalizado" los datos, podemos proceder a extraer los datos de entrenamiento y de validación.

Nuestros datos de validación serán 10000 para ser igual al conjunto de prueba.

Finalmente, creamos una tanda con un tamaño de tanda igual al total de muestras de validación.

In [12]:
X_validacion = X_entreno_normalizado[-num_obs_validacion: , : , : ]
y_validacion = y_entreno[-num_obs_validacion:]

Similarmente, los datos de entrenamiento son todos los demás por lo que nos salteamos tantas observaciones como las hay en el conjunto de validación.

In [13]:
X_entreno = X_entreno_normalizado[ : X_entreno_normalizado.shape[0] - num_obs_validacion, : , : ]
y_entreno = y_entreno[ : y_entreno.shape[0] - num_obs_validacion]
num_obs_entreno = y_entreno.shape[0]

Convertir de Arreglos Numpy a Tensores

In [14]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

Barajear y hacer tandas con el conjunto de datos de entrenamiento

In [15]:
TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)

Hacer tandas con los conjunto de validación y prueba, no se necesita barajearlos

In [16]:
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

## Modelo

### Delineamos el modelo

Cuando pensamos sobre un algoritmo de aprenzaje profundo, casi siempre solo lo imaginamos.  Asi que esta vez, hagámoslo.  :)

In [17]:
tamanio_entrada = 784
tamanio_salida = 10

Usaremos el mismo ancho para ambas capas escondidas.  No es una necesidad!

In [18]:
tamanio_capa_escondida = 50

# Definimos cómo se verá el modelo

La primera capa (la de entrada):  cada observación es de 28x28 píxeles, por lo tanto es un tensor de rango 2.

Como aún no hemos aprendido sobre CNNs, no sabemos como alimentar este tipo de entrada a nuestra red, por lo tanto hay que "aplanar" las imágenes.  Hay un método conveniente **Flatten** que toma nuestro tensor de 28x28 y lo convierte en  un vector (None), o (784,)...porque 28x28 = 784.  Esto nos permite crear una red de alimentación hacia adelante.

    
**tf.keras.layers.Dense** básicamente implementa:  output = activation(dot(entrada, peso) + sesgo).  Requiere varios argumentos, pero los más importantes para nosotros son el ancho de la capa escondida y la función de activación.

La capa final no es diferente, solo nos aseguramos de activarla con **softmax**


In [19]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

c:\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Seleccionar el optimizador y la función de pérdida

Definimos el optimizador que nos gustaría utilizar, la función de pérdida, y las métricas que nos interesa obtener en cada interacción

In [20]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Entrenamiento

Acá es donde entrenamos el modelo que hemos construído

Determinamos el número máximo de épocas.

Ajustamos el modelo , especificando:

* los datos de entrenamiento
* el número total de épocas
* y los datos de validación que creamos en el formato (entradas, metas)

In [21]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 1s - 3ms/step - accuracy: 0.8741 - loss: 0.4405 - val_accuracy: 0.9370 - val_loss: 0.2234
Epoch 2/5
500/500 - 1s - 1ms/step - accuracy: 0.9409 - loss: 0.1987 - val_accuracy: 0.9549 - val_loss: 0.1601
Epoch 3/5
500/500 - 1s - 1ms/step - accuracy: 0.9553 - loss: 0.1528 - val_accuracy: 0.9601 - val_loss: 0.1415
Epoch 4/5
500/500 - 1s - 1ms/step - accuracy: 0.9625 - loss: 0.1257 - val_accuracy: 0.9627 - val_loss: 0.1255
Epoch 5/5
500/500 - 1s - 1ms/step - accuracy: 0.9683 - loss: 0.1074 - val_accuracy: 0.9627 - val_loss: 0.1266


## Probar el modelo

Como se discutió en clase, luego del entrenamiento (con los datos de entrenamiento), y la validación (con los datos de validación), probamos el potencial de predicción final de nuestro modelo con el conjunto de datos de prueba que el algoritmo NUNCA ha visto antes.

Es muy importante reconocer que estar "jugando" con los hiperparámetros sobre-ajusta el conjunto de datos de validación.

La prueba es la instancia absolutamente final. Nunca debe probarse el modelo antes de haber completamente ajustado el modelo.

Si se ajusta el modelo después de hacer la prueba, se empezará a sobre-ajustar el conjunto de datos de prueba, que echaría "por los suelos" el propósito original del mismo.

In [22]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9586 - loss: 20.3336 


In [23]:
# Si se desea, se puede aplicar un formateo "bonito"
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 18.86. Precisión de prueba: 96.38%


Utilizando el modelo inicial y los hiperparámetros dados en este notebook, la precisión de prueba final debe ser aproximadamente 97%.

Cada vez que se ejecuta el código, se obtiene una precisión diferente debido a la "barajeada" de las tandas, los pesos se inicializan en forma diferente, etc.

Finalmente, intencionalmente se ha llegado a una solución subóptima, para que puedan tener la oportunidad de mejorarla como ejercicio de laboratorio.

### 1.

In [25]:
tamanio_capa_escondida = 200
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 2s - 4ms/step - accuracy: 0.9167 - loss: 0.2887 - val_accuracy: 0.9619 - val_loss: 0.1315
Epoch 2/5
500/500 - 1s - 3ms/step - accuracy: 0.9674 - loss: 0.1087 - val_accuracy: 0.9713 - val_loss: 0.0986
Epoch 3/5
500/500 - 1s - 2ms/step - accuracy: 0.9769 - loss: 0.0737 - val_accuracy: 0.9713 - val_loss: 0.0900
Epoch 4/5
500/500 - 1s - 2ms/step - accuracy: 0.9851 - loss: 0.0500 - val_accuracy: 0.9749 - val_loss: 0.0876
Epoch 5/5
500/500 - 1s - 2ms/step - accuracy: 0.9879 - loss: 0.0380 - val_accuracy: 0.9762 - val_loss: 0.0812


In [26]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9703 - loss: 18.2626  
Pérdida de prueba: 15.57. Precisión de prueba: 97.46%


In [33]:
tamanio_capa_escondida = 250
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 2s - 4ms/step - accuracy: 0.9219 - loss: 0.2701 - val_accuracy: 0.9654 - val_loss: 0.1192
Epoch 2/5
500/500 - 1s - 3ms/step - accuracy: 0.9698 - loss: 0.1019 - val_accuracy: 0.9728 - val_loss: 0.0861
Epoch 3/5
500/500 - 1s - 3ms/step - accuracy: 0.9793 - loss: 0.0667 - val_accuracy: 0.9759 - val_loss: 0.0788
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9847 - loss: 0.0472 - val_accuracy: 0.9757 - val_loss: 0.0779
Epoch 5/5
500/500 - 1s - 3ms/step - accuracy: 0.9902 - loss: 0.0315 - val_accuracy: 0.9779 - val_loss: 0.0803
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9721 - loss: 15.4772 
Pérdida de prueba: 14.31. Precisión de prueba: 97.57%


### 2.

In [36]:
tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 1s - 3ms/step - accuracy: 0.8717 - loss: 0.4352 - val_accuracy: 0.9403 - val_loss: 0.2116
Epoch 2/5
500/500 - 1s - 1ms/step - accuracy: 0.9462 - loss: 0.1813 - val_accuracy: 0.9546 - val_loss: 0.1591
Epoch 3/5
500/500 - 1s - 1ms/step - accuracy: 0.9587 - loss: 0.1357 - val_accuracy: 0.9614 - val_loss: 0.1340
Epoch 4/5
500/500 - 1s - 1ms/step - accuracy: 0.9658 - loss: 0.1123 - val_accuracy: 0.9650 - val_loss: 0.1288
Epoch 5/5
500/500 - 1s - 1ms/step - accuracy: 0.9702 - loss: 0.0956 - val_accuracy: 0.9650 - val_loss: 0.1205
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.9614 - loss: 20.9527
Pérdida de prueba: 18.59. Precisión de prueba: 96.61%


### 3.

In [42]:
tamanio_capa_escondida = 250
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 4rta capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 5nta capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 3s - 6ms/step - accuracy: 0.9181 - loss: 0.2709 - val_accuracy: 0.9641 - val_loss: 0.1162
Epoch 2/5
500/500 - 2s - 4ms/step - accuracy: 0.9672 - loss: 0.1098 - val_accuracy: 0.9677 - val_loss: 0.1080
Epoch 3/5
500/500 - 2s - 4ms/step - accuracy: 0.9764 - loss: 0.0766 - val_accuracy: 0.9706 - val_loss: 0.1062
Epoch 4/5
500/500 - 2s - 4ms/step - accuracy: 0.9816 - loss: 0.0590 - val_accuracy: 0.9714 - val_loss: 0.1003
Epoch 5/5
500/500 - 2s - 4ms/step - accuracy: 0.9852 - loss: 0.0475 - val_accuracy: 0.9750 - val_loss: 0.0913
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9711 - loss: 15.8529 
Pérdida de prueba: 13.74. Precisión de prueba: 97.61%


### 4.

In [46]:
tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5


500/500 - 1s - 2ms/step - accuracy: 0.7707 - loss: 1.0394 - val_accuracy: 0.9042 - val_loss: 0.4194
Epoch 2/5
500/500 - 1s - 1ms/step - accuracy: 0.9068 - loss: 0.3555 - val_accuracy: 0.9277 - val_loss: 0.2669
Epoch 3/5
500/500 - 1s - 1ms/step - accuracy: 0.9270 - loss: 0.2599 - val_accuracy: 0.9393 - val_loss: 0.2167
Epoch 4/5
500/500 - 1s - 1ms/step - accuracy: 0.9391 - loss: 0.2135 - val_accuracy: 0.9483 - val_loss: 0.1893
Epoch 5/5
500/500 - 1s - 1ms/step - accuracy: 0.9478 - loss: 0.1822 - val_accuracy: 0.9545 - val_loss: 0.1686
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.9384 - loss: 0.2074
Pérdida de prueba: 0.19. Precisión de prueba: 94.44%


### 5.

In [47]:
tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 1s - 2ms/step - accuracy: 0.8879 - loss: 0.4130 - val_accuracy: 0.9438 - val_loss: 0.1944
Epoch 2/5
500/500 - 1s - 1ms/step - accuracy: 0.9483 - loss: 0.1729 - val_accuracy: 0.9618 - val_loss: 0.1388
Epoch 3/5
500/500 - 1s - 1ms/step - accuracy: 0.9622 - loss: 0.1268 - val_accuracy: 0.9655 - val_loss: 0.1179
Epoch 4/5
500/500 - 1s - 1ms/step - accuracy: 0.9698 - loss: 0.1012 - val_accuracy: 0.9672 - val_loss: 0.1117
Epoch 5/5
500/500 - 1s - 1ms/step - accuracy: 0.9748 - loss: 0.0848 - val_accuracy: 0.9699 - val_loss: 0.0986
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step - accuracy: 0.9578 - loss: 0.1469
Pérdida de prueba: 0.13. Precisión de prueba: 96.27%


### 6.

In [49]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 10000
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
5/5 - 1s - 158ms/step - accuracy: 0.2238 - loss: 2.2078 - val_accuracy: 0.3492 - val_loss: 2.0312
Epoch 2/5
5/5 - 0s - 35ms/step - accuracy: 0.3925 - loss: 1.9267 - val_accuracy: 0.4927 - val_loss: 1.7454
Epoch 3/5
5/5 - 0s - 37ms/step - accuracy: 0.5364 - loss: 1.6387 - val_accuracy: 0.6272 - val_loss: 1.4420
Epoch 4/5
5/5 - 0s - 37ms/step - accuracy: 0.6534 - loss: 1.3472 - val_accuracy: 0.7258 - val_loss: 1.1513
Epoch 5/5
5/5 - 0s - 36ms/step - accuracy: 0.7374 - loss: 1.0803 - val_accuracy: 0.7917 - val_loss: 0.9043
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.7863 - loss: 41.8038
Pérdida de prueba: 41.80. Precisión de prueba: 78.63%


### 7.

In [50]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 1
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
50000/50000 - 38s - 760us/step - accuracy: 0.9212 - loss: 0.2655 - val_accuracy: 0.9483 - val_loss: 0.1846
Epoch 2/5
50000/50000 - 37s - 744us/step - accuracy: 0.9548 - loss: 0.1592 - val_accuracy: 0.9593 - val_loss: 0.1583
Epoch 3/5
50000/50000 - 37s - 745us/step - accuracy: 0.9622 - loss: 0.1409 - val_accuracy: 0.9601 - val_loss: 0.1661
Epoch 4/5
50000/50000 - 38s - 761us/step - accuracy: 0.9655 - loss: 0.1305 - val_accuracy: 0.9597 - val_loss: 0.1859
Epoch 5/5
50000/50000 - 38s - 755us/step - accuracy: 0.9688 - loss: 0.1239 - val_accuracy: 0.9640 - val_loss: 0.1960
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 5s 479us/step - accuracy: 0.9528 - loss: 63.9141
Pérdida de prueba: 55.19. Precisión de prueba: 95.83%


In [51]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 1
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
50000/50000 - 31s - 612us/step - accuracy: 0.9126 - loss: 0.2838 - val_accuracy: 0.9452 - val_loss: 0.1803
Epoch 2/5
50000/50000 - 28s - 553us/step - accuracy: 0.9571 - loss: 0.1442 - val_accuracy: 0.9584 - val_loss: 0.1369
Epoch 3/5
50000/50000 - 28s - 568us/step - accuracy: 0.9663 - loss: 0.1113 - val_accuracy: 0.9665 - val_loss: 0.1217
Epoch 4/5
50000/50000 - 29s - 572us/step - accuracy: 0.9709 - loss: 0.0936 - val_accuracy: 0.9638 - val_loss: 0.1267
Epoch 5/5
50000/50000 - 28s - 568us/step - accuracy: 0.9755 - loss: 0.0808 - val_accuracy: 0.9723 - val_loss: 0.1081
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 5s 462us/step - accuracy: 0.9607 - loss: 26.8626
Pérdida de prueba: 22.06. Precisión de prueba: 96.63%


### 8. 

In [52]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

learning_rate = 0.0001
optimizador = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compilar el modelo con el optimizador personalizado
modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 1s - 2ms/step - accuracy: 0.6855 - loss: 1.2770 - val_accuracy: 0.8748 - val_loss: 0.5524
Epoch 2/5
500/500 - 1s - 1ms/step - accuracy: 0.8833 - loss: 0.4580 - val_accuracy: 0.9089 - val_loss: 0.3479
Epoch 3/5
500/500 - 1s - 1ms/step - accuracy: 0.9068 - loss: 0.3423 - val_accuracy: 0.9196 - val_loss: 0.2891
Epoch 4/5
500/500 - 1s - 1ms/step - accuracy: 0.9185 - loss: 0.2958 - val_accuracy: 0.9270 - val_loss: 0.2587
Epoch 5/5
500/500 - 1s - 1ms/step - accuracy: 0.9247 - loss: 0.2677 - val_accuracy: 0.9318 - val_loss: 0.2384
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.9139 - loss: 34.4986
Pérdida de prueba: 30.74. Precisión de prueba: 92.74%


### 9.

In [53]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

learning_rate = 0.02
optimizador = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compilar el modelo con el optimizador personalizado
modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
500/500 - 1s - 2ms/step - accuracy: 0.9056 - loss: 0.3160 - val_accuracy: 0.9464 - val_loss: 0.1905
Epoch 2/5
500/500 - 1s - 1ms/step - accuracy: 0.9406 - loss: 0.2089 - val_accuracy: 0.9476 - val_loss: 0.1940
Epoch 3/5
500/500 - 1s - 1ms/step - accuracy: 0.9500 - loss: 0.1863 - val_accuracy: 0.9470 - val_loss: 0.2027
Epoch 4/5
500/500 - 1s - 1ms/step - accuracy: 0.9555 - loss: 0.1628 - val_accuracy: 0.9550 - val_loss: 0.1807
Epoch 5/5
500/500 - 1s - 1ms/step - accuracy: 0.9578 - loss: 0.1524 - val_accuracy: 0.9365 - val_loss: 0.2193
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - accuracy: 0.9211 - loss: 48.9121
Pérdida de prueba: 43.08. Precisión de prueba: 93.12%


### 10.

In [79]:
# Definir los datos
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

TAMANIO_TANDA = 150  # Aumentar el tamaño del lote
datos_entreno = datos_entreno.shuffle(buffer_size=num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

tamanio_capa_escondida = 250


modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 4rta capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 5nta capa escondida

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 6xta capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 7ima capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
learning_rate = 0.003
optimizador = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compilar el modelo con el optimizador personalizado
modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
334/334 - 5s - 16ms/step - accuracy: 0.9061 - loss: 0.3182 - val_accuracy: 0.9473 - val_loss: 0.2066
Epoch 2/5
334/334 - 3s - 9ms/step - accuracy: 0.9610 - loss: 0.1433 - val_accuracy: 0.9682 - val_loss: 0.1247
Epoch 3/5
334/334 - 3s - 9ms/step - accuracy: 0.9708 - loss: 0.1110 - val_accuracy: 0.9701 - val_loss: 0.1153
Epoch 4/5
334/334 - 4s - 11ms/step - accuracy: 0.9777 - loss: 0.0884 - val_accuracy: 0.9722 - val_loss: 0.1118
Epoch 5/5
334/334 - 5s - 15ms/step - accuracy: 0.9798 - loss: 0.0779 - val_accuracy: 0.9757 - val_loss: 0.1063
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9661 - loss: 17.4262
Pérdida de prueba: 15.72. Precisión de prueba: 97.15%
